[View in Colaboratory](https://colab.research.google.com/github/joaoflf/handson-ml/blob/master/ch11_ex10.ipynb)

Restart VM

In [0]:
#!kill -9 -1

Imports and fetch tensorboard plugin

In [2]:
!git clone https://github.com/mixuala/colab_utils.git
from tensorflow.examples.tutorials.mnist import input_data
import os
import colab_utils.tboard
import tensorflow as tf
import numpy as np
from functools import partial
from datetime import datetime
from random import choice

Cloning into 'colab_utils'...
remote: Counting objects: 219, done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 219 (delta 16), reused 43 (delta 13), pack-reused 171
Receiving objects: 100% (219/219), 60.62 KiB | 7.58 MiB/s, done.
Resolving deltas: 100% (84/84), done.


Setup logs directory and tensorboard

In [3]:
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
ROOT = %pwd
root_log_dir = os.path.join(ROOT, 'tf_logs')
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=root_log_dir )

calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= http://193501ae.ngrok.io


'http://193501ae.ngrok.io'

Build neural nets

In [0]:
tf.reset_default_graph()

n_inputs = 28 * 28
learning_rate = 0.01
momentum = 0.95

he_init = tf.contrib.layers.variance_scaling_initializer()
training = tf.placeholder_with_default(False, shape=(), name='training')

def create_hidden_layers(name, input_layer, no_layers, layer_size=100, activation_function=tf.nn.elu, initilalizer=he_init):
    for i in range(1, no_layers+1):
      hidden = tf.layers.dense(input_layer, layer_size, name=name+"/hidden"+str(i), kernel_initializer=initilalizer, activation=activation_function)
      input_layer = hidden
    return input_layer
  
X_a = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X_a')
X_b = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X_b')
y = tf.placeholder(tf.int32, shape=(None, 1), name='y')

dnn_a = create_hidden_layers('dnn_a', X_a, 5)
dnn_b = create_hidden_layers('dnn_b', X_b, 5)

concat_layer = tf.layers.dense(tf.concat([dnn_a, dnn_b], 1), 10, name='dnn_concat', kernel_initializer=he_init, activation=tf.nn.elu)
logits = tf.layers.dense(concat_layer, 1, name='logits', kernel_initializer=he_init)
y_prob = tf.nn.sigmoid(logits)
y_pred = tf.cast(tf.greater_equal(logits, 0), tf.int32)

with tf.name_scope('loss'):
  y_as_float = tf.cast(y, tf.float32)
  xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_as_float, logits=logits)
  loss = tf.reduce_mean(xentropy, name='loss')

with tf.name_scope('train'):
  optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
  training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
  y_pred_correct = tf.equal(y_pred, y)
  accuracy = tf.reduce_mean(tf.cast(y_pred_correct, tf.float32), name='accuracy')


loss_summary = tf.summary.scalar('Loss', loss)
init = tf.global_variables_initializer()
saver = tf.train.Saver()


Create function to generate batches of mnist image pairs

In [5]:
mnist = input_data.read_data_sets('/tmp/data/')
mnist_train_images = mnist.train.images
mnist_train_labels = mnist.train.labels
                                 
def generate_pairs_batch(X, y, batch_size):
  X_pair_1 = []
  X_pair_2 = []
  y_pair = []

  
  while len(X_pair_1) < batch_size // 2:
    idx1, idx2 = np.random.randint(0, len(X)-1, 2)
    if (idx1 != idx2 and y[idx1] == y[idx2]):
      X_pair_1.append(X[idx1])
      X_pair_2.append(X[idx2])
      y_pair.append([0])
      
  while len(X_pair_1) < batch_size:
    idx, idx2 = np.random.randint(0, len(X)-1, 2)
    if (idx1 != idx2 and y[idx1] != y[idx2]):
      X_pair_1.append(X[idx1])
      X_pair_2.append(X[idx2])
      y_pair.append([1])
      
  rnd_indices = np.random.permutation(batch_size)
      
  return np.array(X_pair_1)[rnd_indices], np.array(X_pair_2)[rnd_indices], np.array(y_pair)[rnd_indices]

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Train neural net

In [6]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "{}/run-{}/".format(root_log_dir, now)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

batch_size = 500
n_epochs = 50
batch_step = 0

with tf.Session() as sess:
  init.run()
  
  for epoch in range(n_epochs):
    for iteration in range(len(mnist_train_images) // batch_size):
        X_batch_a, X_batch_b, y_batch = generate_pairs_batch(mnist_train_images, mnist_train_labels, batch_size)
        sess.run(training_op, feed_dict={X_a: X_batch_a, X_b: X_batch_b, y: y_batch})
        batch_step+=1
    summary_str, acc_train, loss_str = sess.run([loss_summary, accuracy, loss], feed_dict={X_a: X_batch_a, X_b: X_batch_b, y: y_batch})
    file_writer.add_summary(summary_str,batch_step)
    print(epoch, "Train accuracy:", acc_train, 'Loss:', loss_str)
    
    if epoch % 5 == 0:
      X_test_a, X_test_b, y_test = generate_pairs_batch(mnist.test.images, mnist.test.labels, len(mnist.test.images))
      acc_test = accuracy.eval(feed_dict={X_a: X_test_a, X_b: X_test_b, y: y_test})
      print(epoch, "Test accuracy:", acc_test)
      
    save_path = saver.save(sess, './digit_compare_model.ckpt')
  file_writer.close()
  sess.close()

0 Train accuracy: 0.264 Loss: 0.70405805
0 Test accuracy: 0.5003
1 Train accuracy: 0.756 Loss: 0.6485502
2 Train accuracy: 0.44 Loss: 0.69411844
3 Train accuracy: 0.5 Loss: 0.69911695
4 Train accuracy: 0.53 Loss: 0.68604296
5 Train accuracy: 0.646 Loss: 0.6537176
5 Test accuracy: 0.6272
6 Train accuracy: 0.7 Loss: 0.55715245
7 Train accuracy: 0.708 Loss: 0.56826293
8 Train accuracy: 0.776 Loss: 0.48485264
9 Train accuracy: 0.874 Loss: 0.33398923
10 Train accuracy: 0.722 Loss: 0.5043106
10 Test accuracy: 0.8544
11 Train accuracy: 0.844 Loss: 0.4161686
12 Train accuracy: 0.902 Loss: 0.3304333
13 Train accuracy: 0.83 Loss: 0.45069027
14 Train accuracy: 0.842 Loss: 0.36861688
15 Train accuracy: 0.956 Loss: 0.1568733
15 Test accuracy: 0.9503
16 Train accuracy: 0.852 Loss: 0.30841693
17 Train accuracy: 0.962 Loss: 0.16562739
18 Train accuracy: 0.834 Loss: 0.37564662
19 Train accuracy: 0.868 Loss: 0.2956013
20 Train accuracy: 0.828 Loss: 0.36544743
20 Test accuracy: 0.8858
21 Train accuracy: 

Build digit prediction NN resuing the comparison NN

In [8]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = create_hidden_layers("dnn_a", X, 5)
frozen_outputs = tf.stop_gradient(dnn_outputs)

logits = tf.layers.dense(dnn_outputs, 10, name='outputs', kernel_initializer=he_init)

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
    
with tf.name_scope('train'):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum, use_nesterov=True)
    training_op = optimizer.minimize(loss, name="training_op")
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1, name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "{}/run-{}/".format(root_log_dir, now)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

batch_size = 500
n_epochs = 50
batch_step = 0

loss_summary = tf.summary.scalar('Loss', loss)
init = tf.global_variables_initializer()

dnn_A_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="dnn_a")
restore_saver = tf.train.Saver(var_list={var.op.name: var for var in dnn_A_vars})
saver = tf.train.Saver()

X_train, y_train = mnist.validation.images, mnist.validation.labels
print(X_train.shape)
  
with tf.Session() as sess:
  init.run()
  restore_saver.restore(sess, "./digit_compare_model.ckpt")
  batch_step = 0
  for epoch in range(n_epochs):
      rnd_idx = np.random.permutation(len(X_train))
      for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
          X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
          sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
          batch_step+=1
      summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
      file_writer.add_summary(summary_str,batch_step)
      if epoch % 10 == 0:
          acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
          print(epoch, "Test accuracy:", acc_test)

  save_path = saver.save(sess, "./my_mnist_model_final.ckpt")
      
  file_writer.close()
  sess.close()

(5000, 784)
INFO:tensorflow:Restoring parameters from ./digit_compare_model.ckpt
0 Test accuracy: 0.9307
10 Test accuracy: 0.954
20 Test accuracy: 0.9572
30 Test accuracy: 0.9575
40 Test accuracy: 0.9573
